<h1><center><font size="25">Forecasting series temporales con Python y Scikitlearn</font></center></h1>

<center><b>Joaquín Amat Rodrigo</b></center>

<center><i>Febrero, 2020</i></center>

Más sobre ciencia de datos: [**cienciadedatos.net**](https://cienciadedatos.net)

## Introducción
<br>

Una [serie temporal](https://es.wikipedia.org/wiki/Serie_temporal) (*time series*) es una sucesión de datos ordenados cronológicamente, espaciados a intervalos iguales o desiguales. 

El proceso de [*Forecasting*](https://en.wikipedia.org/wiki/Forecasting) consiste en predecir el valor futuro de una serie temporal, bien modelando la serie temporal únicamente en función de su comportamiento pasado (autorregresivo) o empleando otras variables externas a la serie temporal.

A lo largo de este documento, se describe cómo utilizar modelos de regresión de **Scikit learn** para realizar *forecasting* sobre series temporales. Se van introduciendo de forma progresiva clases y funciones que automatizan los procesos necesarios para entrenar, predecir y selecionar modelos de *forecasting*.

Al final del documento se encuentran las clases y funciones completas, con todas las funcionalidades mostradas.
<br><br>

## Forecasting
<br>

Cuando se trabaja con series temporales, raramente se necesita predecir el siguiente elemento de la serie ($t_{+1}$), sino todo un intervalo futuro o un punto alejado en el tiempo ($t_{+n}$). 

Dado que, para predecir el momento $t_{+2}$ se necesita el valor de $t_{+1}$, y $t_{+1}$ se desconoce, es necesario hacer predicciones recursivas en las que, cada nueva predicción, se basa en la predicción anterior. A este proceso se le conoce *recursive forecasting* o *multi-step forecasting* y es la principal diferencia respecto a los problemas de regresión convencionales.

In [ ]:
import gif
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.random.seed(1234)
y = 10 * np.sin(np.arange(0, 10, 0.1))
y = y + np.random.normal(loc=0.0, scale=4, size = len(y))
datetime = pd.date_range(start='1/1/2020', periods=len(y), freq='D')
datos = pd.Series(y, index=datetime)

# fig, ax = plt.subplots(figsize=(9, 4))
# datos[:-30].plot(ax=ax, c='c', marker='o', label='pasado',  zorder=3)
# datos[-31:].plot(ax=ax, c='gray', label='futuro')
# datos[-30:-15].plot(ax=ax, c='red', marker='o', linestyle='',
#                        label='Valores predichos')
# datos[[-15]].plot(ax=ax, c='red', marker='X', linestyle='', markersize=12,
#                        label='Valor a predecir')
# ax.set_title('Predecir un intervalo a futuro')
# ax.legend();

frames = []

@gif.frame
def custom_plot(i, j):
    fig, ax = plt.subplots(figsize=(9, 4))
    datos[:-30].plot(ax=ax, c='c', marker='o', label='pasado',  zorder=3)
    datos[-31:].plot(ax=ax, c='gray', label='futuro')
    
    datos.iloc[len(datos)-30:i+1].plot(ax=ax, c='red', marker='o', linestyle='',
                                       label='valor predicho')
    
    datos.iloc[[i]].plot(ax=ax, c='red', marker='X', linestyle='',
                         markersize=12,label='valor a predecir')
    ax.set_title(f'Forecasting recursivo (multi-step): (t+{j})')
    ax.legend()
    
    return(ax)

j=1
for i in range(len(datos)-30, len(datos)):
    frame = custom_plot(i, j)
    frames.append(frame)
    j+=1
    
gif.save(frames, './images/forecasting_multi-step.gif', duration=10, unit="s", between="startend")

In /home/ubuntu/miniconda3/envs/cienciadedatos/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ubuntu/miniconda3/envs/cienciadedatos/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ubuntu/miniconda3/envs/cienciadedatos/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/ubuntu/miniconda3/envs/cienciadedatos/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

![forecasting-python](./images/forecasting_multi-step.gif "forecasting-python")

## Forecasting autorregresivo con scikit learn

La principal adaptación que se necesita hacer para aplicar modelos de [scikit learn](https://www.cienciadedatos.net/documentos/py06_machine_learning_python_scikitlearn.html) a problemas de *forecasting* es transformar la serie temporal
en un matriz en la que, cada valor, está asociado a la ventana temporal (lags) que le preceden.

![forecasting-python](./images/transform_timeseries.gif "forecasting-python")

In [ ]:
from IPython.core.display import HTML
HTML("<center><font size='2.5'> <i>" +
"Tranformación de una serie temporal en una matriz de lags y un vector con el valor de la serie que sigue a cada fila de la matriz." +
     "</i></font></center>")

## Ejemplo

### Librerías, Clases y Funciones
<br>

Además de las librerías necesarias, se incluyen la clase `Forecaster` que contine las funcionalidades necesarias para adaptar cualquier modelo de regresón de **Scikit learn** a problemas de *forecasting*.

In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Clase Forecaster
# ==============================================================================
import numpy as np
import pandas as pd
import logging
import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import ParameterGrid


logging.basicConfig(
    format = '%(asctime)-5s %(name)-15s %(levelname)-8s %(message)s', 
    level  = logging.INFO, # Nivel de los eventos que muestra el logger
)

class Forecaster():
    '''
    Convierte un regresor de scikitlearn en un forecaster recursivo (multi-step).
    
    Parameters
    ----------
    regressor : scikitlearn regressor
        Modelo de regresión de scikitlearn.
        
    n_lags : int
        Número de lags utilizados cómo predictores.

    
    Attributes
    ----------
    regressor : scikitlearn regressor
        Modelo de regresión de scikitlearn.
        
    last_lags : array-like, shape (1, n_lags)
        Último valor de lags que ha visto el Forecaster al ser entrenado.
        Se corresponde con el valor de los lags necesarios para predecir el
        siguiente `step` después de los datos de entrenamiento.
     
    '''
    
    def __init__(self, regressor, n_lags):
        
        self.regressor = regressor
        self.n_lags    = n_lags
        self.last_lags = None
        
    
    def create_lags(self, y):
        '''
        Convierte una serie temporal en una matriz donde, cada valor de `y`,
        está asociado a los lags temporales que le preceden.
        
        Parameters
        ----------        
        y : array-like, shape (nº observaciones,)
            Serie temporal de entrenamiento.

        Returns 
        -------
        X_data : array-like, shape (nº observaciones, n_lags)
            Matriz con el valor de los lags.
        
        y_data : array-like, shape (nº observaciones - n_lags)
            Valores de la variable respuesta
            
        '''

        # Número de divisiones
        n_splits = len(y) - self.n_lags

        # Matriz donde almacenar los valore de cada lag
        X_data  = np.full(shape=(n_splits, self.n_lags), fill_value=np.nan, dtype= float)
        y_data  = np.full(shape=(n_splits, 1), fill_value=np.nan, dtype= float)

        for i in range(n_splits):

            train_index = np.array(y.index[i : self.n_lags + i])
            test_index  = y.index[self.n_lags + i]

            X_data[i, :] = y[train_index]
            y_data[i]    = y[test_index]

        return X_data, y_data

        
    def fit(self, y):
        '''
        Entrenamiento del Forecaster
        
        Parameters
        ----------        
        y : array-like, shape (nº observaciones,)
            Serie temporal de entrenamiento.

        Returns 
        -------
        self : object
               objeto Forecaster entrenado
        
        '''
        
        X_train, y_train = self.create_lags(y=y)
        
        self.regressor.fit(X=X_train, y=y_train)
        
        self.last_lags = np.hstack((X_train[-1, 1:], y_train[-1])).reshape(1, -1) 
        
            
    def predict(self, steps, X=None):
        '''
        Predicción recursiva en la que, cada nueva predicción, se utiliza como
        predictor de la siguiente predicción.
        
        Parameters
        ----------
               
        steps : int
            Número de pasos a futuro que se predicen.
            
        X : array-like, shape (1, n_lags), default `None`
            Valor de los predictores con los que se inicia el proceso iterativo
            de predicción. Es decir, el valor de los lags para t+1.
    
            Si `X==None`, se utilizan como predictores iniciales los valores
            almacenados en `self.last_lags`, y las predicciones se inician a 
            continuación de los datos de entrenamiento.

        Returns 
        -------
        predicciones : np.array, shape (steps,)
            Predicciones del modelo.
            
        '''
        
        if X is None:
            X = self.last_lags

        predicciones = np.full(shape=steps, fill_value=np.nan)

        for i in range(steps):

            prediccion = self.regressor.predict(X).ravel()[0]
            predicciones[i] = prediccion

            # Actualizar valores de X con la nueva predicción
            X = X.flatten()
            X = np.append(X[1:], prediccion)
            X = X.reshape(1, -1)


        return(predicciones)
    
    
    def set_params(self, **params):
        '''
        Asignar nuevos valores a los parámetros del modelo scikitlearn contenido
        en el Forecaster.
        
        Parameters
        ----------
        params : dict
            Valor de los parámetros.

        Returns 
        -------
        self
        
        '''
        
        self.regressor.set_params(**params)
        
        
    def set_n_lags(self, n_lags):
        '''
        Asignar nuevo valor al atributo `n_lags` del Forecaster.
        
        Parameters
        ----------
        n_lags : int
            Número de lags utilizados cómo predictores.

        Returns 
        -------
        self
        
        '''
        
        self.n_lags = n_lags
        
        

# Funciones auxiliares para validación de modelos y selección de hiperparámetros
# ==============================================================================  

def time_series_spliter(y, initial_train_size, steps, allow_incomplete_fold=True):
    '''
    
    Dividir los índices de una serie temporal en múltiples pares de train-test.
    Se mantiene el orden temporal de los datos y se incrementa en cada iteración
    el conjunto de entrenamiento.
    
    Parameters
    ----------        
    y : array-like, shape (nº observaciones,)
            Serie temporal de entrenamiento. 
    
    initial_train_size: int 
        Número de observaciones de entrenamiento en la partición inicial.
        
    steps : int
        Número de pasos a futuro que se predicen.
        
    allow_incomplete_fold : bool, default `True`
        Se permite que el último conjunto de test esté incompleto si este
        no alcanza `steps` observaciones. De lo contrario, se descartan las
        últimas observaciones.

    Yields
    ------
    train : ndarray
        Índices del conjunto de entrenamiento.
        
    test : ndarray
        Índices del conjunto de test.
        
    '''
    
  
    folds     = (len(y) - initial_train_size) // steps  + 1
    remainder = (len(y) - initial_train_size) % steps    
    
    for i in range(folds):
          
        if i < folds - 1:
            train_end     = initial_train_size + i * steps    
            train_indices = range(train_end)
            test_indices  = range(train_end, train_end + steps)
            
        else:
            if remainder != 0 and allow_incomplete_fold:
                train_end     = initial_train_size + i * steps  
                train_indices = range(train_end)
                test_indices  = range(train_end, train_end + remainder)
            else:
                break
        
        yield train_indices, test_indices
        

def ts_cv_forecaster(forecaster, y, initial_train_size, steps, metric,
                     allow_incomplete_fold=True):
    '''
    Validación cruzada de un objeto Forecaster, manteniendo el orden temporal de
    los datos e incrementando en cada iteración el conjunto de entrenamiento.
    
    Parameters
    ----------
    forecaster : object 
        Objeto de clase Forecaster.
        
    y : array-like, shape (nº observaciones)
            Serie temporal de entrenamiento. 
    
    initial_train_size: int 
        Número de observaciones de entrenamiento utilizadas en la primera iteración
        de validación.
        
    steps : int
        Número de pasos a futuro que se predicen.
        
    allow_incomplete_fold : bool, default `False`
        Se permite que el último conjunto de test esté incompleto si este
        no alcanza `steps` observaciones. De lo contrario, se descartan las
        últimas observaciones.
        
    metric : {'neg_mean_squared_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error'}
        Métrica utilizada para cuantificar la bondad de ajuste del modelo.

    Returns 
    -------
    ts_cv_results: 
        Valor de la métrica obtenido en cada partición.

    '''
    
    ts_cv_results = []
    
    metrics = {
        'neg_mean_squared_error': mean_squared_error,
        'neg_mean_absolute_error': mean_absolute_error,
        'neg_mean_absolute_percentage_error': mean_absolute_percentage_error
    }
    
    metric = metrics[metric]
    
    splits = time_series_spliter(
                y                     = y,
                initial_train_size    = initial_train_size,
                steps                 = steps,
                allow_incomplete_fold = allow_incomplete_fold
             )
    
    for train_index, test_index in splits:
        
        forecaster.fit(y=y.iloc[train_index])      
        pred = forecaster.predict(steps=len(test_index))
               
        metric_value = metric(
                            y_true = y.iloc[test_index],
                            y_pred = pred
                       )
        
        ts_cv_results.append(metric_value)
                          
    return np.array(ts_cv_results)


def grid_search_forecaster(forecaster, y, param_grid, initial_train_size, steps,
                           metric, n_lags_grid=None, allow_incomplete_fold=False,
                           return_best=True):
    '''
    Búsqueda exhaustiva sobre los hiperparámetros de un Forecaster mediante
    validación cruzada.
    
    Parameters
    ----------
    forecaster : object 
        Objeto de clase Forecaster.
        
    y : array-like, shape (nº observaciones,)
        Serie temporal de entrenamiento. 
    
    param_grid : dict
        Valores de hiperparámetros sobre los que hacer la búsqueda
    
    initial_train_size: int 
        Número de observaciones de entrenamiento utilizadas en la primera iteración
        de validación.
        
    steps : int
        Número de pasos a futuro que se predicen.
        
    metric : {'neg_mean_squared_error'}
        Métrica utilizada para cuantificar la bondad de ajuste del modelo.
        
    n_lags_grid : array-like
        Valores de `n_lags` sobre los que hacer la búsqueda.
        
    allow_incomplete_fold : bool, default `False`
        Se permite que el último conjunto de test esté incompleto si este
        no alcanza `steps` observaciones. De lo contrario, se descartan las
        últimas observaciones.
        
    return_best : devuelve modifica el Forecaster y lo entrena con los mejores
                  hiperparámetros encontrados.

    Returns 
    -------
    results: pandas.DataFrame
        Valor de la métrica obtenido en cada combinación de hiperparámetros.

    '''
    n_lags_list = []
    params_list = []
    metric_list = []
    
    if n_lags_grid is None:
        n_lags_grid = [forecaster.n_lags]
    
    param_grid =  list(ParameterGrid(param_grid))
    
    for n_lags in tqdm.tqdm(n_lags_grid, desc='loop n_lags_grid', position=0):
        
        forecaster.set_n_lags(n_lags)
        
        for params in tqdm.tqdm(param_grid, desc='loop param_grid', position=1, leave=False):

            forecaster.set_params(**params)

            cv_metrics = ts_cv_forecaster(
                            forecaster = forecaster,
                            y          = y,
                            initial_train_size = initial_train_size,
                            steps  = steps,
                            metric = metric
                        )
            
            n_lags_list.append(n_lags)
            params_list.append(params)
            metric_list.append(cv_metrics.mean())
            
    resultados = pd.DataFrame({
                    'n_lags': n_lags_list,
                    'params': params_list,
                    'metric': metric_list})
    
    resultados = resultados.sort_values(by='metric', ascending=True)
    
    if return_best:
        best_n_lags = resultados['n_lags'].iloc[0]
        best_params = resultados['params'].iloc[0]
        logging.info("Entrenando Forecaster con los mejores parámetros encontrados:")
        logging.info(f"n_lags: {best_n_lags}, params: {best_params}")
        forecaster.set_n_lags(best_n_lags)
        forecaster.set_params(**best_params)
        forecaster.fit(y=y, exog=exog)
            
    return resultados 

### Datos
<br>

Los datos empleados en los ejemplos de este document se han obtenido del magnifico libro [Forecasting: Principles and Practice by Rob J Hyndman and George Athanasopoulos]("https://otexts.com/fpp2/"). Representan el gasto mensual (millones de dolares) en fármacos con corticoides que tuvo el sistema de salud en Australia entre 1991 y 2008.

In [ ]:
# Descarga de datos
# ==============================================================================
url = 'https://raw.githubusercontent.com/JoaquinAmatRodrigo/' \
      + 'Estadistica-machine-learning-python/master/data/elec.csv'
datos = pd.read_csv(url, sep=',')

La columna *fecha* se ha almacenado como `strings`. Para convertirla en `Datetime`, se emplea la función `pd.to_datetime()`. Una vez en formato `Datetime`, y para hacer uso de las funcionalidades de pandas, se establece la columna *fecha* como índice. Además, dado que los datos son mensuales, se indica la frecuencia (*Monthly Started 'MS'*).

In [ ]:
# Preparación del dato
# ==============================================================================
datos['fecha'] = pd.to_datetime(datos['fecha'], format='%Y/%m/%d')
datos = datos.set_index('fecha')
datos = datos.rename(columns={'x': 'y'})
datos = datos.asfreq('MS')
datos = datos['y']
datos = datos.sort_index()

# Si el set de datos tiene valores NAN
#datos.asfreq(freq='MS', fill_value=np.nan)

Se utilizan los últimos 36 meses como conjunto de test para evaluar la capacidad predictiva del modelo.

In [ ]:
# Separación datos train-test
# ==============================================================================
steps = 36
datos_train = datos[:-steps]
datos_test  = datos[-steps:]

fig, ax=plt.subplots(figsize=(9, 4))
datos_train.plot(ax=ax, label='train')
datos_test.plot(ax=ax, label='test')
ax.legend();

### Forecaster
<br>

Se crea y entrena un modelo `Forecaster` a partir de un regresor `RandomForestRegressor` y ventana temporal de 12 lags.

In [ ]:
# Crear y entrenar forecaster
# ==============================================================================

forecaster_rf = Forecaster(
                    regressor=RandomForestRegressor(random_state=123),
                    n_lags=12
                )

forecaster_rf.fit(y=datos_train)

### Predicciones
<br>

Una vez entrenado el modelo, se predicen los datos de test.

In [ ]:
# Predicciones
# ==============================================================================
predicciones = forecaster_rf.predict(steps=steps)
# Se añade el índice a las predicciones
predicciones = pd.Series(data=predicciones, index=datos_test.index)
predicciones.head()

In [ ]:
# Gráfico
# ==============================================================================
fig, ax=plt.subplots(figsize=(9, 4))
datos_train.plot(ax=ax, label='train')
datos_test.plot(ax=ax, label='test')
predicciones.plot(ax=ax, label='predicciones')
ax.legend();

### Error predictores
<br>

Se cuentifica el error que comete el modelo al predecir los valores d ela serie temporal a futuro.

In [ ]:
# Error
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test,
                y_pred = predicciones
            )
print(f"Error de test (mse) {error_mse}")

### Tuning del modelo
<br>

El forecaster entrenado hasta el momento, ha utilizado una ventana temporal de 12 lags y un modelo Random Forest con los hiperparámetros por defecto. Sin embargo, no hay ninguna razón por la que estos valores sean los adecuados.

Con el objetivo de identificar la mejor combinación de lags e hiperparámetros, se recurre a validación cruzada temporal.

In [ ]:
# Grid search de hiperparámetros
# ==============================================================================
forecaster_rf = Forecaster(
                    regressor=RandomForestRegressor(random_state=123),
                    n_lags=12
                 )

param_grid = {'n_estimators': [100, 500, 1000],
              'max_depth': [3, 5, 10]}

n_lags_grid = [5, 12, 20]

resultados_grid = grid_search_forecaster(
                        forecaster  = forecaster_rf,
                        y           = datos_train,
                        param_grid  = param_grid,
                        n_lags_grid = n_lags_grid,
                        steps       = 10,
                        metric      = 'neg_mean_squared_error',
                        initial_train_size    = int(len(datos_train)*0.5),
                        allow_incomplete_fold = False
                    )

In [ ]:
# Resultados Grid Search
# ==============================================================================
resultados_grid

Los mejores resultados se obtienen utilizando una ventana temporal de 20 lags y una configuración de random forest {'max_depth': 10, 'n_estimators': 1000}.

### Modelo final
<br> 

Se entrena de nuevo un Forecaster con la configuración optima encontrada por validación cruzada.

In [ ]:
# Crear y entrenar forecaster
# ==============================================================================
regressor = RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=123)

forecaster_rf = Forecaster(
                    regressor=regressor,
                    n_lags=20
                )

forecaster_rf.fit(y=datos_train)

In [ ]:
# Predicciones
# ==============================================================================
forecaster_rf.fit(y=datos_train)
predicciones = forecaster_rf.predict(steps=steps)
# Se añade el índice a las predicciones
predicciones = pd.Series(data=predicciones, index=datos_test.index)
predicciones.head()

In [ ]:
# Gráfico
# ==============================================================================
fig, ax=plt.subplots(figsize=(9, 4))
datos_train.plot(ax=ax, label='train')
datos_test.plot(ax=ax, label='test')
predicciones.plot(ax=ax, label='predicciones')
ax.legend();

In [ ]:
# Error
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test,
                y_pred = predicciones
            )
print(f"Error de test (mse) {error_mse}")

## Forecasting con variables exógenas

Supóngase que se dispone de información de otra variable, cuyo valor a futuro se conoce, y se quiere utilizar como predictor.

## Ejemplo
<br>

### Clases y funciones

In [ ]:
import numpy as np
import pandas as pd
import logging
import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import ParameterGrid


logging.basicConfig(
    format = '%(asctime)-5s %(name)-15s %(levelname)-8s %(message)s', 
    level  = logging.INFO, # Nivel de los eventos que se registran en el logger
)

class Forecaster():
    '''
    Combierte un regresor de scikitlearn en un forecaster recursivo (multi-step).
    
    Parameters
    ----------
    regressor : scikitlearn regressor
        Modelo de regresión de scikitlearn.
        
    n_lags : int
        Número de lags utilizados cómo predictores.

    
    Attributes
    ----------
    regressor : scikitlearn regressor
        Modelo de regresión de scikitlearn.
        
    last_lags : array-like, shape (1, n_lags)
        Último valor de lags que ha visto el Forecaster al ser entrenado. Se
        corresponde con el valor de los lags necesarios para predecir el
        siguiente `step` después de los datos de entrenamiento.
        
    included_exog: bool, default `False`.
        Si el Forecaster se ha entrenado utilizando variable/s exógenas.
     
    '''
    
    def __init__(self, regressor, n_lags):
        
        self.regressor     = regressor
        self.n_lags        = n_lags
        self.last_lags     = None
        self.included_exog = False
        
    
    def create_lags(self, y):
        '''
        Combierte una serie temporal en una matriz donde, cada valor de y,
        está asociado a los lags temporales que le preceden.
        
        Parameters
        ----------        
        y : array-like, shape (nº observaviones)
            Serie temporal de entrenamiento.

        Returns 
        -------
        X_data : array-like, shape (nº observaviones, n_lags)
            Matriz con el valor de los lags.
        
        y_data : array-like, shape (nº observaviones - n_lags)
            Valores de la variable respuesta.
            
        '''

        # Número de divisiones
        n_splits = len(y) - self.n_lags

        # Matriz donde almacenar los valore de cada lag
        X_data  = np.full(shape=(n_splits, self.n_lags), fill_value=np.nan, dtype= float)
        y_data  = np.full(shape=(n_splits, 1), fill_value=np.nan, dtype= float)

        for i in range(n_splits):

            train_index = np.array(y.index[i : self.n_lags + i])
            test_index  = y.index[self.n_lags + i]

            X_data[i, :] = y[train_index]
            y_data[i]    = y[test_index]

        return X_data, y_data

        
    def fit(self, y, exog=None):
        '''
        Entrenamiento del Forecaster.
        
        Parameters
        ----------        
        y : array-like, shape (nº observaviones)
            Serie temporal de entrenamiento.
            
        exog : array-like, shape (nº observaviones, )
            Variable/s exógena a la serie temporal que se quiere incluir como
            predictor/es.

        Returns 
        -------
        self : object
               objeto Forecaster entrenado.
        
        '''       
        
        X_train, y_train = self.create_lags(y=y)
        
        if exog is not None:
            self.included_exog = True
            self.regressor.fit(
                # Se tiene que eliminar de exog las primeras self.n_lags
                # posiciones ya que son NaN en X_train.
                X=np.column_stack((X_train, exog[self.n_lags:])),
                y=y_train
            )
        else:
            self.regressor.fit(X=X_train, y=y_train)
        
        # Se guarda la última ventana temporal de lags de entrenamiento para que 
        # pueda utilizarse como predictores en la primera iteración del predict().
        self.last_lags = np.hstack((X_train[-1, 1:], y_train[-1])).reshape(1, -1) 
        
            
    def predict(self, steps, X=None, exog=None):
        '''
        Proceso iterativo en la que, cada predicción, se utiliza como
        predictor de la siguiente predicción.
        
        Parameters
        ----------
               
        steps : int
            Número de pasos a futuro que se predicen.
            
        X : array-like, shape (1, n_lags)
            Valor de los predictores con los que se inicia el proceso iterativo
            de predicción. Es decir, el valor de los lags para t+1.
    
            Si `X==None`, se utilizan como predictores iniciales los valores
            almacenados en `self.last_lags`, y las predicciones se inician a 
            continuación de los datos de entrenamiento.
            
        exog : array-like, shape (nº observaviones, )
            Variable/s exógena a la serie temporal que se quiere incluir como
            predictor/es.

        Returns 
        -------
        predicciones : np.array, shape (nº observaciones)
            Predicciones del modelo.
            
        '''

        if exog is None and self.included_exog:
            raise Exception(
                "Forecaster entrenado con variable/s exogena/s. Se tiene " \
                + "que aportar esta misma variable en el predict()."
            )
                
        
        if X is None:
            X = self.last_lags
            
        if exog is not None:
            if isinstance(exog, (pd.Series, list)):
                exog = np.array(exog).reshape(-1,1)
            

        predicciones = np.full(shape=steps, fill_value=np.nan)

        for i in range(steps):
            if exog is None:
                prediccion = self.regressor.predict(X=X)
            else:
                prediccion = self.regressor.predict(X=np.column_stack((X, exog[i])))
                
            predicciones[i] = prediccion.ravel()[0]

            # Actualizar valores de X. Se descarta la primera posición de X y se
            # añade la nueva predicción al final.
            X = X.flatten()
            X = np.append(X[1:], prediccion)
            X = X.reshape(1, -1)


        return predicciones
    
    
    def set_params(self, **params):
        '''
        Asignar nuevos valores a los parámetros del modelo scikitlearn contenido
        en el Forecaster.
        
        Parameters
        ----------
        params : dict
            Valor de los parámetros del modelo.

        Returns 
        -------
        self
        
        '''
        
        self.regressor.set_params(**params)
        
        
    def set_n_lags(self, n_lags):
        '''
        Asignar nuevo valor al atributo `n_lags` del Forecaster.
        
        Parameters
        ----------
        n_lags : int
            Número de lags utilizados como predictores.

        Returns 
        -------
        self
        
        '''
        
        self.n_lags = n_lags
        
        
def time_series_spliter(y, initial_train_size, steps, allow_incomplete_fold=True):
    '''
    
    Dividir los índices de una serie temporal en múltiples pares train-test.
    Se mantiene el orden temporal de los datos y se incrementa en cada iteración
    el conjunto de entrenamiento.
    
    Parameters
    ----------        
    y : array-like, shape (nº observaviones)
            Serie temporal de entrenamiento. 
                
    initial_train_size: int 
        Número de observaciones de entrenamiento utilizadas en la primera iteración
        de validación.
        
    steps : int
        Número de pasos a futuro que se predicen.
        
    allow_incomplete_fold : bool, default `True`
        Se permite que el último conjunto de test esté incompleto si este
        no alcanza `steps` observaciones. De lo contrario, se descartan las
        últimas observaciones.

    Yields
    ------
    train : ndarray
        Índices del conjunto de entrenamiento.
        
    test : ndarray
        Índices del conjunto de test.
        
        
    Notes
    -----
    
    Los índices devueltos equivalen a la posición (empezando por cero), no a los
    índices que tenga la serie temporal `y`.
    '''
    
  
    folds     = (len(y) - initial_train_size) // steps  + 1
    remainder = (len(y) - initial_train_size) % steps    
    
    for i in range(folds):
          
        if i < folds - 1:
            train_end     = initial_train_size + i * steps    
            train_indices = range(train_end)
            test_indices  = range(train_end, train_end + steps)
            
        else:
            if remainder != 0 and allow_incomplete_fold:
                train_end     = initial_train_size + i * steps  
                train_indices = range(train_end)
                test_indices  = range(train_end, train_end + remainder)
            else:
                break
        
        yield train_indices, test_indices
        

def ts_cv_forecaster(forecaster, y, initial_train_size, steps, metric, exog=None,
                     allow_incomplete_fold=True):
    '''
    Validación cruzada de un Forecaster, manteniendo el orden temporal de los
    datos e incrementando en cada iteración el conjunto de entrenamiento `steps`
    posiciones.
    
    Parameters
    ----------
    forecaster : object 
        Objeto de clase Forecaster.
        
    y : array-like, shape (nº observaviones)
        Serie temporal de entrenamiento. 
            
    exog : array-like, shape (nº observaviones, )
        Variable/s exógena a la serie temporal que se quiere incluir como
        predictor/es.
    
    initial_train_size: int 
        Número de observaciones de entrenamiento utilizadas en la primera iteración
        de validación.
        
    steps : int
        Número de pasos a futuro que se predicen.
        
    allow_incomplete_fold : bool, default `False`
        Se permite que el último conjunto de test esté incompleto si este
        no alcanza `steps` observaciones. De lo contrario, se descartan las
        últimas observaciones.
        
    metric : {'neg_mean_squared_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error'}
        Métrica utilizada para cuantificar la bondad de ajuste del modelo.

    Returns 
    -------
    ts_cv_results: 
        Valor de la métrica obtenido en cada partición.

    '''
    
    ts_cv_results = []
    
    metrics = {
        'neg_mean_squared_error': mean_squared_error,
        'neg_mean_absolute_error': mean_absolute_error,
        'neg_mean_absolute_percentage_error': mean_absolute_percentage_error
    }
    
    metric = metrics[metric]
    
    splits = time_series_spliter(
                y                     = y,
                initial_train_size    = initial_train_size,
                steps                 = steps,
                allow_incomplete_fold = allow_incomplete_fold
             )
    
    for train_index, test_index in splits:
        
        if exog is None:
            forecaster.fit(y=y.iloc[train_index])      
            pred = forecaster.predict(steps=len(test_index))
        else:
            forecaster.fit(y=y.iloc[train_index], exog=exog.iloc[train_index])      
            pred = forecaster.predict(steps=len(test_index), exog=exog.iloc[test_index])
               
        metric_value = metric(
                            y_true = y.iloc[test_index],
                            y_pred = pred
                       )
        
        ts_cv_results.append(metric_value)
                          
    return np.array(ts_cv_results)


def grid_search_forecaster(forecaster, y, param_grid, initial_train_size, steps,
                           metric, exog=None, n_lags_grid=None,
                           allow_incomplete_fold=False, return_best=True):
    '''
    Busqueda exaustiva sobre los hiperparámetros de un Forecaster mediante
    validación cruzada.
    
    Parameters
    ----------
    forecaster : object 
        Objeto de clase Forecaster.
        
    y : array-like, shape (nº observaviones)
        Serie temporal de entrenamiento. 
        
    exog : array-like, shape (nº observaviones, )
            Variable/s exógena a la serie temporal que se quiere incluir como
            predictor.
    
    param_grid : dict
        Valores de hiperparámetros sobre los que hacer la búsqueda
    
    initial_train_size: int 
        Número de observaciones de entrenamiento utilizadas en la primera iteración
        de validación.
        
    steps : int
        Número de pasos a futuro que se predicen.
        
    metric : {'neg_mean_squared_error'}
        Métrica utilizada para cuantificar la bondad de ajuste del modelo.
        
    n_lags_grid : array-like
        Valores de `n_lags` sobre los que hacer la búsqueda.
        
    allow_incomplete_fold : bool, default `False`
        Se permite que el ultimo conjunto de test esté incompleto si este
        no alcanza `steps` observaciones. De lo contrario, se descartan las
        últimas observaciones.
    
    return_best : devuelve modifica el Forecaster y lo entrena con los mejores
                  hiperparámetros encontrados.
    Returns 
    -------
    results: pandas.DataFrame
        Valor de la métrica obtenido en cada combinación de hiperparámetros.

    '''
    n_lags_list = []
    params_list = []
    metric_list = []
    
    if n_lags_grid is None:
        n_lags_grid = [forecaster.n_lags]
    
    param_grid =  list(ParameterGrid(param_grid))
    
    for n_lags in tqdm.tqdm(n_lags_grid, desc='loop n_lags_grid', position=0):
        
        forecaster.set_n_lags(n_lags)
        
        for params in tqdm.tqdm(param_grid, desc='loop param_grid', position=1, leave=False):

            forecaster.set_params(**params)

            cv_metrics = ts_cv_forecaster(
                            forecaster = forecaster,
                            y          = y,
                            exog       = exog,
                            initial_train_size = initial_train_size,
                            steps  = steps,
                            metric = metric
                         )
            
            n_lags_list.append(n_lags)
            params_list.append(params)
            metric_list.append(cv_metrics.mean())
            
    resultados = pd.DataFrame({
                    'n_lags': n_lags_list,
                    'params': params_list,
                    'metric': metric_list})
    
    resultados = resultados.sort_values(by='metric', ascending=True)
    
    if return_best:
        best_n_lags = resultados['n_lags'].iloc[0]
        best_params = resultados['params'].iloc[0]
        logging.info("Entrenando Forecaster con los mejores parámetros encontrados:")
        logging.info(f"n_lags: {best_n_lags}, params: {best_params}")
        forecaster.set_n_lags(best_n_lags)
        forecaster.set_params(**best_params)
        forecaster.fit(y=y, exog=exog)
            
    return resultados 

### Datos

In [ ]:
# Descarga de datos
# ==============================================================================
url = 'https://raw.githubusercontent.com/JoaquinAmatRodrigo/' \
      + 'Estadistica-machine-learning-python/master/data/h2o.csv'
datos = pd.read_csv(url, sep=',')

# Preparación del dato
# ==============================================================================
datos['fecha'] = pd.to_datetime(datos['fecha'], format='%Y/%m/%d')
datos = datos.set_index('fecha')
datos = datos.rename(columns={'x': 'y'})
datos = datos.asfreq('MS')
datos = datos['y']
datos = datos.sort_index()

In [ ]:
# Datos
# ==============================================================================
datos_exog = datos.rolling(window=10, closed='right').mean() + 0.5
datos_exog = datos_exog[10:]
datos = datos[10:]

fig, ax=plt.subplots(figsize=(9, 4))
datos.plot(ax=ax, label='y')
datos_exog.plot(ax=ax, label='variable exógena')
ax.legend();

In [ ]:
# Separación datos train-test
# ==============================================================================
steps = 36
datos_train = datos[:-steps]
datos_test  = datos[-steps:]

datos_exog_train = datos_exog[:-steps]
datos_exog_test  = datos_exog[-steps:]

### Forecaster

In [ ]:
# Crear y entrenar forecaster
# ==============================================================================
forecaster_rf = Forecaster(
                    regressor=RandomForestRegressor(random_state=123),
                    n_lags=12
                )

forecaster_rf.fit(y=datos_train, exog=datos_exog_train)

### Predicciones

In [ ]:
# Predicciones
# ==============================================================================
predicciones = forecaster_rf.predict(steps=steps, exog=datos_exog_test)
# Se añade el índice a las predicciones
predicciones = pd.Series(data=predicciones, index=datos_test.index)

In [ ]:
# Gráfico
# ==============================================================================
fig, ax=plt.subplots(figsize=(9, 4))
datos_train.plot(ax=ax, label='train')
datos_test.plot(ax=ax, label='test')
predicciones.plot(ax=ax, label='predicciones')
ax.legend();

### Error predictores

In [ ]:
# Error
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test,
                y_pred = predicciones
            )
print(f"Error de test (mse) {error_mse}")

### Tuning del modelo

In [ ]:
# Grid search de hiperparámetros
# ==============================================================================
forecaster_rf = Forecaster(
                    regressor=RandomForestRegressor(random_state=123),
                    n_lags=12
                 )

param_grid = {'n_estimators': [50, 100, 500],
              'max_depth': [3, 5, 10]}

n_lags_grid = [5, 12, 20]

resultados_grid = grid_search_forecaster(
                        forecaster  = forecaster_rf,
                        y           = datos_train,
                        exog        = datos_exog_train,
                        param_grid  = param_grid,
                        n_lags_grid = n_lags_grid,
                        steps       = 10,
                        metric      = 'neg_mean_squared_error',
                        initial_train_size    = int(len(datos_train)*0.5),
                        allow_incomplete_fold = False,
                        return_best = True
                    )

In [ ]:
# Resultados Grid Search
# ==============================================================================
resultados_grid

Los mejores resultados se obtienen utilizando una ventana temporal de 20 lags y una configuración de random forest {'max_depth': 3, 'n_estimators': 50}.

### Modelo final

Se entrena de nuevo un Forecaster con la configuración óptima encontrada por validación cruzada. Si se indica `return_best = True` en el `grid_search_forecaster()`, se modifica *inplace* el forecaster y se entrena con la mejor combinación de parámetros encontrada.

In [ ]:
# Predicciones
# ==============================================================================
predicciones = forecaster_rf.predict(steps=steps, exog=datos_exog_test)
# Se añade el índice a las predicciones
predicciones = pd.Series(data=predicciones, index=datos_test.index)

In [ ]:
# Gráfico
# ==============================================================================
fig, ax=plt.subplots(figsize=(9, 4))
datos_train.plot(ax=ax, label='train')
datos_test.plot(ax=ax, label='test')
predicciones.plot(ax=ax, label='predicciones')
ax.legend();

In [ ]:
# Error
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test,
                y_pred = predicciones
            )
print(f"Error de test (mse) {error_mse}")

## Forecasting con predictores custom

### Clases y funciones

In [ ]:
import numpy as np
import pandas as pd
import logging
import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import ParameterGrid


logging.basicConfig(
    format = '%(asctime)-5s %(name)-15s %(levelname)-8s %(message)s', 
    level  = logging.INFO, # Nivel de los eventos que se registran en el logger
)

class Forecaster():
    '''
    Combierte un regresor de scikitlearn en un forecaster recursivo (multi-step).
    
    Parameters
    ----------
    regressor : scikitlearn regressor
        Modelo de regresión de scikitlearn.
        
    n_lags : int, default 1
        Número de lags utilizados cómo predictores.
        
    predictor_transformer: function, default `None`
        Función que recibe una serie temporal y devuelve una matriz de predictores
        y el valor de la serie temporal asociado. Si `None` se utilizan como
        predictores los primeros `n_lags` lags.
        
    window_size: int, default `None`
        Tamaño de la ventana temporal a pasado que necesita `predictor_transformer`
        para crear los predictores.

    
    Attributes
    ----------
    regressor : scikitlearn regressor
        Modelo de regresión de scikitlearn.
        
    last_lags : array-like, shape (1, n_lags)
        Último valor de lags que ha visto el Forecaster al ser entrenado. Se
        corresponde con el valor de los lags necesarios para predecir el
        siguiente `step` después de los datos de entrenamiento.
        
    included_exog: bool, default `False`.
        Si el Forecaster se ha entrenado utilizando variable/s exógenas.
        
    predictor_transformer: function, default `None`
        Función que recibe una serie temporal y devuelve una matriz de predictores
        y el valor de la serie temporal asociado.  Si `None` se utilizan como
        predictores los primeros `n_lags` lags.
        
    window_size: int, default `None`
        Tamaño de la ventana temporal a pasado que necesita `predictor_transformer`
        para crear los predictores.
     
    '''
    
    def __init__(self, regressor, n_lags=1, predictor_transformer=None,
                 window_size=None):
        
        self.regressor = regressor
        self.n_lags    = n_lags
        self.last_lags = None
        self.included_exog = False
        self.predictor_transformer = predictor_transformer
        self.window_size = window_size
        
        if self.predictor_transformer is not None and self.window_size is None:
            raise Exception(
                'Es necesario indicar el window_size si se utiliza predictor_transformer.'
            )
        
    
    def create_lags(self, y):
        '''
        Combierte una serie temporal en una matriz donde, cada valor de y,
        está asociado a los lags temporales que le preceden.
        
        Parameters
        ----------        
        y : array-like, shape (nº observaviones)
            Serie temporal de entrenamiento.

        Returns 
        -------
        X_data : array-like, shape (nº observaviones, n_lags)
            Matriz con el valor de los lags.
        
        y_data : array-like, shape (nº observaviones - n_lags)
            Valores de la variable respuesta.
            
        '''

        # Número de divisiones
        n_splits = len(y) - self.n_lags

        # Matriz donde almacenar los valore de cada lag
        X_data  = np.full(shape=(n_splits, self.n_lags), fill_value=np.nan, dtype= float)
        y_data  = np.full(shape=(n_splits, 1), fill_value=np.nan, dtype= float)

        for i in range(n_splits):

            train_index = np.array(y.index[i : self.n_lags + i])
            test_index  = y.index[self.n_lags + i]

            X_data[i, :] = y[train_index]
            y_data[i]    = y[test_index]

        return X_data, y_data
    
    
    def create_predictors(self, y):
        '''
        Creación de predictores a partir de `y`. Si el Forecaster no tiene definido
        una función `predictor_transformer`, se utilizan como predictores los `n_lags`
        primeros lags.
        
        Parameters
        ----------        
        y : array-like, shape (nº observaviones)
            Serie temporal de entrenamiento.

        Returns 
        -------
        X_data : array-like, shape (nº observaviones, )
            Matriz con el valor de los predictores.
        
        y_data : array-like, shape (nº observaviones)
            Variable respuesta
            
        '''
        
        if self.predictor_transformer is None:
            return self.create_lags(y=y)
        else:
            return self.predictor_transformer(y=y)

        
    def fit(self, y, exog=None):
        '''
        Entrenamiento del Forecaster.
        
        Parameters
        ----------        
        y : array-like, shape (nº observaviones)
            Serie temporal de entrenamiento.
            
        exog : array-like, shape (nº observaviones, )
            Variable/s exógena a la serie temporal que se quiere incluir como
            predictor/es.

        Returns 
        -------
        self : object
               objeto Forecaster entrenado.
        
        '''       
        
        X_train, y_train = self.create_predictors(y=y)
        
        if exog is not None:
            self.included_exog = True
            self.regressor.fit(
                # Se tiene que eliminar de exog las primeras self.n_lags
                # posiciones ya que son NaN en X_train.
                X=np.column_stack((X_train, exog[self.n_lags:])),
                y=y_train
            )
        else:
            self.regressor.fit(X=X_train, y=y_train)
        
        if self.predictor_transformer is None:
            # Se guarda la última ventana temporal de lags de entrenamiento para que 
            # pueda utilizarse como predictores en la primera iteración del predict().
            self.last_lags = np.hstack((X_train[-1, 1:], y_train[-1])).reshape(1, -1)
            print(self.last_lags)
        
        else:
            # Se guarda la última ventana temporal de `y` del entrenamiento para 
            # utilizarla en la creación de predictores en la primera iteración
            # del predict().
            self.last_window = y_train[-self.window_size:].flatten()
            print(self.last_window)
            
        
            
    def predict(self, steps, X=None, y=None, exog=None):
        '''
        Proceso iterativo en la que, cada predicción, se utiliza como
        predictor de la siguiente predicción.
        
        Parameters
        ----------
               
        steps : int
            Número de pasos a futuro que se predicen.
            
        X : array-like, shape (1, n_lags)
            Valor de los predictores con los que se inicia el proceso iterativo
            de predicción. Es decir, el valor de los lags para t+1.
    
            Si `X=None`, se utilizan como predictores iniciales los valores
            almacenados en `self.last_lags`, y las predicciones se inician a 
            continuación de los datos de entrenamiento.
        
        y : array-like, shape (self.window_size)
            Cuando se utiliza `predictor_transformer`, y es la ventana temporal
            de la serie `y` necesaria para crear los predictores. Si `y=None`,
            se utilizan los valores almacenados en `self.last_window`.
            
        exog : array-like, shape (nº observaviones, )
            Variable/s exógena a la serie temporal que se quiere incluir como
            predictor/es.

        Returns 
        -------
        predicciones : np.array, shape (nº observaciones)
            Predicciones del modelo.
            
        '''

        if exog is None and self.included_exog:
            raise Exception(
                "Forecaster entrenado con variable/s exogena/s. Se tiene " \
                + "que aportar esta misma variable en el predict()."
            )
            
        if y is not None and self.predictor_transformer is not None and len(y) < self.window_size:
            raise Exception(
                f"predictor_transformer requiere que `y` tenga como mínimo"
                f"de {self.window_size} observaciones."
            )
            
        if exog is not None:
            if isinstance(exog, (pd.Series, list)):
                exog = np.array(exog).reshape(-1,1)
            

        predicciones = np.full(shape=steps, fill_value=np.nan)

        if self.predictor_transformer is None:
            
            if X is None:
                X = self.last_lags
            
            for i in range(steps):
                if exog is None:
                    prediccion = self.regressor.predict(X=X)
                else:
                    prediccion = self.regressor.predict(X=np.column_stack((X, exog[i])))

                predicciones[i] = prediccion.ravel()[0]

                # Actualizar valores de X. Se descarta la primera y se añade la
                # nueva predicción al final.
                X = X.flatten()
                X = np.append(X[1:], prediccion)
                X = X.reshape(1, -1)
                
        else:
            
            if y is None:
                last_window = self.last_window.copy()
            else:
                last_window = y.copy()
                        
            for i in range(steps):
                # La última observación de last_window debe formar parte de los
                # en el cálculo de predictores. Como por defecto create_predictors()
                # separa la última predicción en y_train. Se añade al final de
                # last_window un valor 0 de relleno.
                last_window = np.append(last_window, 0)
                X, _ = self.create_predictors(last_window)
                X = X[-1].reshape(1, -1)
                if exog is None:
                    prediccion = self.regressor.predict(X=X)
                else:
                    prediccion = self.regressor.predict(X=np.column_stack((X, exog[i])))

                predicciones[i] = prediccion.ravel()[0]

                # Actualizar valores de last_window. Se descarta la primera y se
                # añade la nueva predicción al final. Se elimina el 0 de relleno
                # que tiene last_window al final.
                last_window = np.append(last_window[1:-1], prediccion)

        return predicciones
    
    
    def set_params(self, **params):
        '''
        Asignar nuevos valores a los parámetros del modelo scikitlearn contenido
        en el Forecaster.
        
        Parameters
        ----------
        params : dict
            Valor de los parámetros del modelo.

        Returns 
        -------
        self
        
        '''
        
        self.regressor.set_params(**params)
        
        
    def set_n_lags(self, n_lags):
        '''
        Asignar nuevo valor al atributo `n_lags` del Forecaster.
        
        Parameters
        ----------
        n_lags : int
            Número de lags utilizados como predictores.

        Returns 
        -------
        self
        
        '''
        
        self.n_lags = n_lags
        
        
def time_series_spliter(y, initial_train_size, steps, allow_incomplete_fold=True):
    '''
    
    Dividir los índices de una serie temporal en múltiples pares train-test.
    Se mantiene el orden temporal de los datos y se incrementa en cada iteración
    el conjunto de entrenamiento.
    
    Parameters
    ----------        
    y : array-like, shape (nº observaviones)
            Serie temporal de entrenamiento. 
                
    initial_train_size: int 
        Número de observaciones de entrenamiento utilizadas en la primera iteración
        de validación.
        
    steps : int
        Número de pasos a futuro que se predicen.
        
    allow_incomplete_fold : bool, default `True`
        Se permite que el último conjunto de test esté incompleto si este
        no alcanza `steps` observaciones. De lo contrario, se descartan las
        últimas observaciones.

    Yields
    ------
    train : ndarray
        Índices del conjunto de entrenamiento.
        
    test : ndarray
        Índices del conjunto de test.
        
        
    Notes
    -----
    
    Los índices devueltos equivalen a la posición (empezando por cero), no a los
    índices que tenga la serie temporal `y`.
    '''
    
  
    folds     = (len(y) - initial_train_size) // steps  + 1
    remainder = (len(y) - initial_train_size) % steps    
    
    for i in range(folds):
          
        if i < folds - 1:
            train_end     = initial_train_size + i * steps    
            train_indices = range(train_end)
            test_indices  = range(train_end, train_end + steps)
            
        else:
            if remainder != 0 and allow_incomplete_fold:
                train_end     = initial_train_size + i * steps  
                train_indices = range(train_end)
                test_indices  = range(train_end, train_end + remainder)
            else:
                break
        
        yield train_indices, test_indices
        

def ts_cv_forecaster(forecaster, y, initial_train_size, steps, metric, exog=None,
                     allow_incomplete_fold=True):
    '''
    Validación cruzada de un Forecaster, manteniendo el orden temporal de los
    datos e incrementando en cada iteración el conjunto de entrenamiento `steps`
    posiciones.
    
    Parameters
    ----------
    forecaster : object 
        Objeto de clase Forecaster.
        
    y : array-like, shape (nº observaviones)
        Serie temporal de entrenamiento. 
            
    exog : array-like, shape (nº observaviones, )
        Variable/s exógena a la serie temporal que se quiere incluir como
        predictor/es.
    
    initial_train_size: int 
        Número de observaciones de entrenamiento utilizadas en la primera iteración
        de validación.
        
    steps : int
        Número de pasos a futuro que se predicen.
        
    allow_incomplete_fold : bool, default `False`
        Se permite que el último conjunto de test esté incompleto si este
        no alcanza `steps` observaciones. De lo contrario, se descartan las
        últimas observaciones.
        
    metric : {'neg_mean_squared_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error'}
        Métrica utilizada para cuantificar la bondad de ajuste del modelo.

    Returns 
    -------
    ts_cv_results: 
        Valor de la métrica obtenido en cada partición.

    '''
    
    ts_cv_results = []
    
    metrics = {
        'neg_mean_squared_error': mean_squared_error,
        'neg_mean_absolute_error': mean_absolute_error,
        'neg_mean_absolute_percentage_error': mean_absolute_percentage_error
    }
    
    metric = metrics[metric]
    
    splits = time_series_spliter(
                y                     = y,
                initial_train_size    = initial_train_size,
                steps                 = steps,
                allow_incomplete_fold = allow_incomplete_fold
             )
    
    for train_index, test_index in splits:
        
        if exog is None:
            forecaster.fit(y=y.iloc[train_index])      
            pred = forecaster.predict(steps=len(test_index))
        else:
            forecaster.fit(y=y.iloc[train_index], exog=exog.iloc[train_index])      
            pred = forecaster.predict(steps=len(test_index), exog=exog.iloc[test_index])
               
        metric_value = metric(
                            y_true = y.iloc[test_index],
                            y_pred = pred
                       )
        
        ts_cv_results.append(metric_value)
                          
    return np.array(ts_cv_results)


def grid_search_forecaster(forecaster, y, param_grid, initial_train_size, steps,
                           metric, exog=None, n_lags_grid=None,
                           allow_incomplete_fold=False, return_best=True):
    '''
    Busqueda exaustiva sobre los hiperparámetros de un Forecaster mediante
    validación cruzada.
    
    Parameters
    ----------
    forecaster : object 
        Objeto de clase Forecaster.
        
    y : array-like, shape (nº observaviones)
        Serie temporal de entrenamiento. 
        
    exog : array-like, shape (nº observaviones, )
            Variable/s exógena a la serie temporal que se quiere incluir como
            predictor.
    
    param_grid : dict
        Valores de hiperparámetros sobre los que hacer la búsqueda
    
    initial_train_size: int 
        Número de observaciones de entrenamiento utilizadas en la primera iteración
        de validación.
        
    steps : int
        Número de pasos a futuro que se predicen.
        
    metric : {'neg_mean_squared_error'}
        Métrica utilizada para cuantificar la bondad de ajuste del modelo.
        
    n_lags_grid : array-like
        Valores de `n_lags` sobre los que hacer la búsqueda.
        
    allow_incomplete_fold : bool, default `False`
        Se permite que el ultimo conjunto de test esté incompleto si este
        no alcanza `steps` observaciones. De lo contrario, se descartan las
        últimas observaciones.
    
    return_best : devuelve modifica el Forecaster y lo entrena con los mejores
                  hiperparámetros encontrados.
    Returns 
    -------
    results: pandas.DataFrame
        Valor de la métrica obtenido en cada combinación de hiperparámetros.

    '''
    n_lags_list = []
    params_list = []
    metric_list = []
    
    if n_lags_grid is None:
        n_lags_grid = [forecaster.n_lags]
    
    param_grid =  list(ParameterGrid(param_grid))
    
    for n_lags in tqdm.tqdm(n_lags_grid, desc='loop n_lags_grid', position=0):
        
        forecaster.set_n_lags(n_lags)
        
        for params in tqdm.tqdm(param_grid, desc='loop param_grid', position=1, leave=False):

            forecaster.set_params(**params)

            cv_metrics = ts_cv_forecaster(
                            forecaster = forecaster,
                            y          = y,
                            exog       = exog,
                            initial_train_size = initial_train_size,
                            steps  = steps,
                            metric = metric
                         )
            
            n_lags_list.append(n_lags)
            params_list.append(params)
            metric_list.append(cv_metrics.mean())
            
    resultados = pd.DataFrame({
                    'n_lags': n_lags_list,
                    'params': params_list,
                    'metric': metric_list})
    
    resultados = resultados.sort_values(by='metric', ascending=True)
    
    if return_best:
        best_n_lags = resultados['n_lags'].iloc[0]
        best_params = resultados['params'].iloc[0]
        print("Entrenando Forecaster con los mejores parámetros encontrados:")
        print(f"n_lags: {best_n_lags}, params: {best_params}")
        forecaster.set_n_lags(best_n_lags)
        forecaster.set_params(**best_params)
        forecaster.fit(y=y, exog=exog)
            
    return resultados 

### Datos

In [ ]:
# Descarga de datos
# ==============================================================================
url = 'https://raw.githubusercontent.com/JoaquinAmatRodrigo/' \
      + 'Estadistica-machine-learning-python/master/data/h2o.csv'
datos = pd.read_csv(url, sep=',')

# Preparación del dato
# ==============================================================================
datos['fecha'] = pd.to_datetime(datos['fecha'], format='%Y/%m/%d')
datos = datos.set_index('fecha')
datos = datos.rename(columns={'x': 'y'})
datos = datos.asfreq('MS')
datos = datos['y']
datos = datos.sort_index()

In [ ]:
# Datos
# ==============================================================================
fig, ax=plt.subplots(figsize=(9, 4))
datos.plot(ax=ax, label='y')
datos_exog.plot(ax=ax, label='variable exógena')
ax.legend();

In [ ]:
# Separación datos train-test
# ==============================================================================
steps = 36
datos_train = datos[:-steps]
datos_test  = datos[-steps:]

datos_exog_train = datos_exog[:-steps]
datos_exog_test  = datos_exog[-steps:]

### Predictores
<br>

Para este ejemplo, además de los 10 primeros lasgs, se añade como predictor una ventana móvil.

In [ ]:
def predictores_custom(y):
    
    # Creación de lags
    X_train = pd.DataFrame({'y':y.copy()})
    for i in range(12, 0, -1):
        X_train[f'lag_{i}'] = X_train['y'].shift(i)
        
    # Media movil 30. El shift es necesario para no incluir el valor actual
    # en el cálculo
    #X_train['roll_mean'] = X_train['y'].rolling(30).mean().shift(1)
    
    X_train = X_train.dropna()
    y_train = X_train['y'].to_numpy()
    X_train = X_train.drop(columns='y').to_numpy()  
    
    return X_train, y_train       
        

### Forecaster

In [ ]:
# Crear y entrenar forecaster
# ==============================================================================
forecaster_rf = Forecaster(
                    regressor   = RandomForestRegressor(random_state=123),
                    n_lags      = 12,
                    predictor_transformer = predictores_custom,
                    window_size = 13
                )

forecaster_rf.fit(y=datos_train)

In [ ]:
predictores_custom(np.array([0.85680282, 1.00159317, 0.99486433, 1.134432,   1.181011,   1.216037,   1.257238,
  1.17069,    0.597639,   0.65259,    0.670505,   0.695248,   0.842263, 0]))

In [ ]:
predictores_custom(np.array([0.85680282, 1.00159317, 0.99486433, 1.134432,   1.181011,   1.216037,   1.257238,
  1.17069,    0.597639,   0.65259,    0.670505,   0.695248,   0.842263, 0]))

### Predicciones

In [ ]:
# Predicciones
# ==============================================================================
predicciones = forecaster_rf.predict(steps=steps)
# Se añade el índice a las predicciones
predicciones = pd.Series(data=predicciones, index=datos_test.index)

In [ ]:
# Gráfico
# ==============================================================================
fig, ax=plt.subplots(figsize=(9, 4))
datos_train.plot(ax=ax, label='train')
datos_test.plot(ax=ax, label='test')
predicciones.plot(ax=ax, label='predicciones')
ax.legend();

### Error predictores

In [ ]:
# Error
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test,
                y_pred = predicciones
            )
print(f"Error de test (mse) {error_mse}")

### Tuning del modelo

In [ ]:
# Grid search de hiperparámetros
# ==============================================================================
forecaster_rf = Forecaster(
                    regressor=RandomForestRegressor(random_state=123),
                    n_lags=12
                 )

param_grid = {'n_estimators': [50, 100, 500],
              'max_depth': [3, 5, 10]}

n_lags_grid = [5, 12, 20]

resultados_grid = grid_search_forecaster(
                        forecaster  = forecaster_rf,
                        y           = datos_train,
                        exog        = datos_exog_train,
                        param_grid  = param_grid,
                        n_lags_grid = n_lags_grid,
                        steps       = 10,
                        metric      = 'neg_mean_squared_error',
                        initial_train_size    = int(len(datos_train)*0.5),
                        allow_incomplete_fold = False
                    )

In [ ]:
# Resultados Grid Search
# ==============================================================================
resultados_grid

Los mejores resultados se obtienen utilizando una ventana temporal de 20 lags y una configuración de random forest {'max_depth': 3, 'n_estimators': 50}.

### Modelo final

Se entrena de nuevo un Forecaster con la configuración optima encontrada por validación cruzada.

In [ ]:
# Crear y entrenar forecaster
# ==============================================================================
regressor = RandomForestRegressor(max_depth=3, n_estimators=50, random_state=123)

forecaster_rf = Forecaster(
                    regressor=regressor,
                    n_lags=20
                )

forecaster_rf.fit(y=datos_train, exog=datos_exog_train)

In [ ]:
# Predicciones
# ==============================================================================
predicciones = forecaster_rf.predict(steps=steps, exog=datos_exog_test)
# Se añade el índice a las predicciones
predicciones = pd.Series(data=predicciones, index=datos_test.index)

In [ ]:
# Gráfico
# ==============================================================================
fig, ax=plt.subplots(figsize=(9, 4))
datos_train.plot(ax=ax, label='train')
datos_test.plot(ax=ax, label='test')
predicciones.plot(ax=ax, label='predicciones')
ax.legend();

In [ ]:
# Error
# ==============================================================================
error_mse = mean_squared_error(
                y_true = datos_test,
                y_pred = predicciones
            )
print(f"Error de test (mse) {error_mse}")

## Información de sesión

In [ ]:
from sinfo import sinfo
sinfo()

## Bibliografía
<br>

Hyndman, R.J., & Athanasopoulos, G. (2018) Forecasting: principles and practice, 2nd edition, OTexts: Melbourne, Australia. [libro](https://otexts.com/fpp2/)

Python Data Science Handbook by Jake VanderPlas [libro](https://www.amazon.es/gp/product/1491912057/ref=as_li_qf_asin_il_tl?ie=UTF8&tag=cienciadedato-21&creative=24630&linkCode=as2&creativeASIN=1491912057&linkId=73620d22f9d4a0a76d27592dabf13c83)

Python for Finance: Mastering Data-Driven Finance [libro](https://www.amazon.es/gp/product/1492024333/ref=as_li_qf_asin_il_tl?ie=UTF8&tag=cienciadedato-21&creative=24630&linkCode=as2&creativeASIN=1492024333&linkId=70c3175ad015970cd1c2328b7a40a055)

[Markus Löning, Anthony Bagnall, Sajaysurya Ganesh, Viktor Kazakov, Jason Lines, Franz Király (2019): “sktime: A Unified Interface for Machine Learning with Time Series”](http://learningsys.org/neurips19/assets/papers/sktime_ml_systems_neurips2019.pdf)

[Markus Löning, Tony Bagnall, Sajaysurya Ganesh, George Oastler, Jason Lines, ViktorKaz, …, Aadesh Deshmukh (2020). alan-turing-institute/sktime. Zenodo. http://doi.org/10.5281/zenodo.3749000](https://github.com/alan-turing-institute/sktime)

In [ ]:
%%html
<style>
.text_cell_render p {
    text-align: justify;
    font-family: 'Open Sans','Helvetica Neue',Helvetica,Arial,sans-serif;
    #font-size: 16px;
    line-height: 1.5;
    font-weight: 400;
    text-shadow: none;
    color: #333333;
    text-rendering: optimizeLegibility;
    letter-spacing: +0.1px;
    margin-bottom: 1.15rem;
    font-size: 1.15em
}

#notebook-container {
    background-color: #fcfcfc;
}

div.inner_cell {
    margin-right: 5%;
}

.output_png {
        display: table-cell;
        text-align: center;
        vertical-align: middle;
}

.rendered_html code {
    background-color: #f2f2f2;
    font-family: monospace;
    color: #a20505;
    font-size: 15px;
    #font-size: 1em;
    padding: 1px 1px;
    border: solid;
    border-color: darkgray;
    border-width: thin;
}

.rendered_html h1 {
    padding-top: 50px;
}

.rendered_html h2 {
    font-size: 30px
    margin-top: 0;
    font-size: 2.488em;
}

.rendered_html h3 {
    font-size: 25px;
}

.rendered_html h4 {
    font-size: 20px;
}

</style>

**¿Cómo citar este documento?**

<p style="text-align:left"><font size="3" color="#555">
Forecasting series temporales con Python y Scikitlearn by Joaquín Amat Rodrigo, available under a Attribution 4.0 International (CC BY 4.0) at https://www.cienciadedatos.net
    
</font></p>

<p><strong>¿Te ha gustado el artículo? Tu ayuda es importante</strong></p>

<p style="text-align:left"><font size="3" color="#555">
Mantener un sitio web tiene unos costes elevados, tu contribución me ayudará a seguir generando contenido divulgativo gratuito. ¡Muchísimas gracias!</font>
<form action="https://www.paypal.com/donate" method="post" target="_top">
  <input type="hidden" name="hosted_button_id" value="6NULYFYDKFTQL" />
  <input type="image" src="https://www.paypalobjects.com/en_US/ES/i/btn/btn_donateCC_LG.gif" border="0" name="submit" title="PayPal - The safer, easier way to pay online!" alt="Donate with PayPal button" />
  <img alt="" border="0" src="https://www.paypal.com/en_ES/i/scr/pixel.gif" width="1" height="1" />
  </form>	
</p>

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />This work by  Joaquín Amat Rodrigo is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.